In [3]:
from selenium import webdriver
from tqdm.notebook import tqdm
import pandas as pd
import json
from time import sleep
import os


In [4]:
driver = webdriver.PhantomJS(r'../phantomjs.exe')

In [5]:
def read_config(config_path):
    with open(config_path, 'r', encoding='UTF-8') as config_file:
        return json.load(config_file)

In [6]:
config = read_config(r'../config.json')

In [7]:
base_url = 'https://www.gismeteo.ru/diary/{}/{}/{}/'
years = range(2018, 2021)
months = range(1, 13)

In [26]:
def get_img(el):
    _ = el.find_elements_by_tag_name('img')
    if len(_) > 0:
        return os.path.basename(_[0].get_attribute('src'))[:-4]
    return '-'

def parse_wind(el):
    text = el.text
    _ = text.split(' ')
    if len(_) > 1:
        direction = _[0]
        wind = _[1]
        return [direction, int(''.join(filter(str.isdigit, wind)))]
    return ['-', 0]

result = []
for areal in tqdm(config['urls']):
    for city in config['urls'][areal]:
        url = config['urls'][areal][city]
        id = url.split('-')[-1][:-1]
        for year in years:
            for month in months:
                url_ = base_url.format(id, year, month)
                driver.get(url_)
                table = driver.find_element_by_css_selector('table')
                for row in table.find_elements_by_css_selector('tr'):
                    cols = row.find_elements_by_tag_name('td')
                    if len(cols) == 0:
                        continue
                    try:
                        data = {
                            'day_temp': int(cols[1].text),
                            'day_pressure': int(cols[2].text),
                            'day_obl': get_img(cols[3]),
                            'day_phen': get_img(cols[4]),
                            'day_dir': parse_wind(cols[5])[0],
                            'day_wind': parse_wind(cols[5])[1],
                            'night_temp': int(cols[6].text),
                            'night_pressure': int(cols[7].text),
                            'night_obl': get_img(cols[8]),
                            'night_phen': get_img(cols[9]),
                            'night_dir': parse_wind(cols[10])[0],
                            'night_wind': parse_wind(cols[10])[1],
                        }
                        data.update({
                            'areal': areal,
                            'city': city,
                            'year': year,
                            'month': month,
                            'day': int(cols[0].text)
                        })
                        result.append(data)
                    except Exception as e:
                        print(e)


KeyboardInterrupt: 

In [23]:
import pandas as pd

dt = pd.DataFrame(result)
dt.to_csv('result.csv', index=False)

In [25]:
pd.read_csv('result.csv')

,day_temp,day_pressure,day_obl,day_phen,day_dir,day_wind,night_temp,night_pressure,night_obl,night_phen,night_dir,night_wind,areal,city,year,month,day
0,-18,768,sun,-,-,0,-22,769,sun,-,-,0,Алтайский край,Барнаул,2018,1,1
1,-16,767,sun,-,СВ,1,-22,767,sun,-,СВ,1,Алтайский край,Барнаул,2018,1,2
2,-15,763,sun,-,С,1,-23,762,sun,-,С,1,Алтайский край,Барнаул,2018,1,3
3,-19,761,sun,-,-,0,-25,762,sun,-,-,0,Алтайский край,Барнаул,2018,1,4
4,-22,764,sun,-,-,0,-29,764,sun,-,-,0,Алтайский край,Барнаул,2018,1,5
5,-23,764,dull,-,ЮЗ,1,-21,764,sun,-,ЮЗ,1,Алтайский край,Барнаул,2018,1,6
6,-17,763,sun,-,-,0,-22,763,sun,-,-,0,Алтайский край,Барнаул,2018,1,7
7,-13,758,sun,-,-,0,-19,756,sun,-,-,0,Алтайский край,Барнаул,2018,1,8
8,-6,753,dull,-,Ю,3,-5,754,dull,snow,Ю,3,Алтайский край,Барнаул,2018,1,9
9,-2,750,dull,-,Ю,4,-2,749,dull,-,Ю,4,Алтайский край,Барнаул,2018,1,10
